In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

/home/vasia/anaconda2/envs/p3env/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


## Question1

In [27]:
sildta = pd.read_csv('data/Reveal_EEO1_for_2016.csv')
sildta = sildta[(sildta.job_category.str.contains('Totals')==False)]
sildta = sildta[(sildta.job_category.str.contains('Previous_totals')==False)]
male_sildta = sildta[sildta['gender']=='male']
male_sildta


,company,year,race,gender,job_category,count
0,23andMe,2016,Latino,male,Executive/Senior officials & Mgrs,0
1,23andMe,2016,Latino,male,First/Mid officials & Mgrs,1
2,23andMe,2016,Latino,male,Professionals,7
3,23andMe,2016,Latino,male,Technicians,0
4,23andMe,2016,Latino,male,Sales workers,0
5,23andMe,2016,Latino,male,Administrative support,0
6,23andMe,2016,Latino,male,Craft workers,0
7,23andMe,2016,Latino,male,operatives,0
8,23andMe,2016,Latino,male,laborers and helpers,0
9,23andMe,2016,Latino,male,Service workers,0
